<a href="https://colab.research.google.com/github/lilywright99/scRNAseqtutorial/blob/main/Day_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Random Forest **(Day 4)**

Generate a Random Forest model to see if genes with high leiden values are able to predict cell type
1.   Select and SMOTE cell type of interest
2.   create training and test set
3.   Apply Boruta on training set
4.   Validate test set to generate AUC and ER

### **1. Load in data**

In [ ]:
exp_data <- read.csv('/content/X.csv', header=FALSE)
metadata <-read.csv('/content/obs.csv')
var_data<- read.csv('/content/var.csv')

rownames(exp_data)<-metadata$CellID
colnames(exp_data)<-var_data$Gene

### **2. Create Metadata and Expression data for the 3 cell types; Secretory Epithelial-1, Secretory Epithelial-2, STIC lesion**

In [ ]:
install.packages("dplyr")
library(dplyr)
metadata_SE1 <- metadata[metadata$Celltype %in% c('Secretory Epithelial-1'),]
exp_data_SE1 <- exp_data[rownames(exp_data) %in% metadata_SE1$CellID,]

metadata_SE2 <- metadata[metadata$Celltype %in% c('Secretory Epithelial-2'),]
exp_data_SE2 <- exp_data[rownames(exp_data) %in% metadata_SE2$CellID,]

metadata_STIC <- metadata[metadata$Celltype %in% c('STIC lesion'),]
exp_data_STIC <- exp_data[rownames(exp_data) %in% metadata_STIC$CellID,]

### **3. Analyse disease stages in each cell type and select a cell type for analysis and the disease stage for prediction**

In [ ]:
install.packages("tibble")
library(tibble)
#random forest to predict disease stage
table(metadata_SE1$Disease_stage)
table(metadata_SE2$Disease_stage)
table(metadata_STIC$Disease_stage)

#subset expression data in SE2 into disease stages
exp_data$CellID <- rownames(exp_data)
exp_data_SE2 <- merge(metadata_SE2[,c('CellID','Disease_stage')],exp_data, by='CellID')
exp_data_SE2$Disease_stage[exp_data_SE2$Disease_stage %in% c('Benign','I','IIIc','IIIc1')] <- c('Rest')
library(tibble)
exp_data_SE2 <- tibble::column_to_rownames(exp_data_SE2, "CellID")


### **4. Smote**


**SMOTE (Synthetic Minority Over-sampling Technique)**: A method used to address the issue of imbalanced datasets in machine learning. It helps balance the classes by creating synthetic examples of the minority class rather than simply duplicating existing ones. This makes the dataset more suitable for training machine learning models, which often perform poorly on imbalanced datasets.

In [ ]:
exp_data_SE2$Disease_stage <- as.factor(exp_data_SE2$Disease_stage)

table(exp_data_SE2$Disease_stage)
exp_data_SE2_smoted <- performanceEstimation::smote(Disease_stage~.,exp_data_SE2, perc.over = 0.5, perc.under =3 )
table(exp_data_SE2_smoted$Disease_stage)

### **5. Random Forest**

Random Forest is a versatile and widely used machine learning algorithm that combines the power of multiple decision trees to create a robust and accurate predictive model. It is a type of ensemble learning, where multiple models are trained and combined to improve performance and reduce overfitting.


**Create a testing and training set from the expression data of a cell type.**

The training set and test set are critical components of the process used to train and evaluate the model. They ensure that the model learns effectively from the data and can generalize well to unseen examples.

**Training Set**: The training set is the subset of the data used to build (train) the random forest model.
- The algorithm uses the training set to create individual decision trees.
- Each tree is trained on a bootstrap sample of the training set, which is a random sample with replacement. This introduces diversity among the trees.
- The training process includes:
Splitting the data into branches based on feature values.
Calculating measures like Gini impurity or information gain to choose the best splits.
- On average, about 2/3 of the training data is used to train each tree, while the remaining is left out. These left-out samples are called the Out-Of-Bag (OOB) data.

**Testing Set**: The test set is a separate subset of the data, not used during the training phase. It is used to evaluate the performance of the random forest on unseen data.

Here, 70% of our dataset is used for the training set and 30% as the test set.

In [ ]:
#Random Forest - training and test set
colnames(exp_data_choice_smoted) <- gsub('-','_',colnames(exp_data_choice_smoted))
selected.samples<-sample(c(1:nrow(exp_data_SE2_smoted)),size = nrow(exp_data_SE2_smoted)*0.7)
train.data<-exp_data_SE2_smoted[selected.samples,]
test.data<-exp_data_SE2_smoted[-selected.samples,]

Carry out Boruta

Boruta: Package to identify which features are most relevant for predicting a target variable.

In [ ]:
install.packages("Boruta")
library(Boruta)
bor<-Boruta(Disease_stage~.,data=train.data)
plot(bor)
bor
bor_confirmed <- data.frame(bor$finalDecision[bor$finalDecision %in% "Confirmed"])
bor_confirmed <- rownames(bor_confirmed)

Random Forest Model

In [ ]:
install.packages("randomForest")
library(randomForest)
set.seed(1)  # Setting seed
rf<-randomForest(Disease_stage~.,data=train.data, ntree = 500)

**Plotting the OOB error rate of the model and gini Coefficient.**

The error rate in a random forest quantifies how often the model makes incorrect predictions. It reflects the model's ability to generalize to unseen data and is a critical metric for evaluating its performance. OOB Error is a built-in mechanism of random forest that acts as an internal validation set without the need for a separate test set.

In [ ]:
install.packages("ggplot2")
library(ggplot2)
oob.error.data <- data.frame(Trees = rep(1:nrow(rf$err.rate), times=3),
                             Type = rep(c("OOB","IV","Rest"), each=nrow(rf$err.rate)),
                             Error = c(rf$err.rate[,"OOB"],
                                       rf$err.rate[,"IV"],
                                       rf$err.rate[,"Rest"]))
ggplot(data = oob.error.data,aes(x=Trees, y=Error))+
  geom_line(aes(color=Type))

# Variable importance plot
varImpPlot(classifier_RF)

**Calculating the Out-of-Bag (OOB) Area under curve (AUC) from the model**

**AUC:** The AUC measures the ability of a random forest classifier to distinguish between classes. The AUC refers to the area under the Receiver Operating Characteristic (ROC) curve, which is a graphical representation of a classifier’s performance across different classification thresholds.

The value of AUC ranges from 0 to 1:
- AUC = 0.5: The model performs no better than random guessing.
- AUC = 1.0: The model perfectly distinguishes between positive and negative classes.
- AUC < 0.5: The model is performing worse than random guessing.

**Receiver Operating Characteristic (ROC)**: For each possible threshold (ranging from 0 to 1), the random forest classifier computes the True Positive Rate and False Positive Rate.
By plotting these values for each threshold, you create the ROC curve.



In [ ]:
install.packages("ROCR")
library(ROCR)
#OOB AUC
pred1=predict(rf,type = "prob")
perf = prediction(as.numeric(pred1[,2]),train.data$Disease_stage)
# 1. Area under curve
auc = performance(perf, "auc")
auc <- as.numeric(auc@y.values)
auc
rf$err.rate[nrow(rf$err.rate),1]
# 2. True Positive and Negative Rate
pred3 = performance(perf, "tpr","fpr")
# 3. Plot the ROC curve
plot(pred3,main="ROC Curve for Random Forest",col=2,lwd=2)
abline(a=0,b=1,lwd=2,lty=2,col="gray")

**Validation Analysis on testing dataset**

In [ ]:
install.packages("caret")
library(caret)
# Validation cohort
pred<-predict(rf,test.data)
confusionMatrix(pred,test.data$Disease_stage)

In [ ]:
#Validation AUC
pred1=predict(rf,newdata = test.data,type = "prob")
perf = prediction(pred1[,2],test.data$Disease_stage)
# 1. Area under curve
auc = performance(perf, "auc")
auc <- as.numeric(auc@y.values)
auc
rf$err.rate[nrow(rf$err.rate),1]
# 2. True Positive and Negative Rate
pred3 = performance(perf, "tpr","fpr")
# 3. Plot the ROC curve
plot(pred3,main="ROC Curve for Random Forest",col=2,lwd=2)
abline(a=0,b=1,lwd=2,lty=2,col="gray")


# **Your Try**

In [ ]:
#view table of disease stages in each cell type
table(metadata_SE1$Disease_stage)
table(metadata_SE2$Disease_stage)
table(metadata_STIC$Disease_stage)

#subset expression data in SE2 into disease stages
exp_data$CellID <- rownames(exp_data)
exp_data_choice <- merge(metadata_choice[,c('CellID','Disease_stage')],exp_data, by='CellID')
exp_data_choice$Disease_stage[exp_data_choice$Disease_stage %in% c('other disease stages aka not your chosen disease stage')] <- c('Rest')

exp_data_choice <- tibble::column_to_rownames(exp_data_choice, "CellID")

In [ ]:
exp_data_choice$Disease_stage <- as.factor(exp_data_choice$Disease_stage)

table(exp_data_choice$Disease_stage)
exp_data_choice_smoted <- performanceEstimation::smote(Disease_stage~.,exp_data_choice, perc.over = , perc.under = )
table(exp_data_choice_smoted$Disease_stage)

In [ ]:
bor<-Boruta(Disease_stage~.,data=train.data)
plot(bor)
bor
bor_confirmed <- data.frame(bor$finalDecision[bor$finalDecision %in% "Confirmed"])
bor_confirmed <- rownames(bor_confirmed)

In [ ]:
set.seed(2)  # Setting seed
rf<-randomForest(Disease_stage~.,data=train.data, ntree = 500)

In [ ]:
oob.error.data <- data.frame(Trees = rep(1:nrow(rf$err.rate), times=3),
                             Type = rep(c("OOB","IV","Rest"), each=nrow(rf$err.rate)),
                             Error = c(rf$err.rate[,"OOB"],
                                       rf$err.rate[,"IV"],
                                       rf$err.rate[,"Rest"]))
ggplot(data = oob.error.data,aes(x=Trees, y=Error))+
  geom_line(aes(color=Type))

# Variable importance plot
varImpPlot(classifier_RF)

In [ ]:
#OOB AUC
pred1=predict(rf,type = "prob")
perf = prediction(as.numeric(pred1[,2]),train.data$Disease_stage)
# 1. Area under curve
auc = performance(perf, "auc")
auc <- as.numeric(auc@y.values)
auc
rf$err.rate[nrow(rf$err.rate),1]
# 2. True Positive and Negative Rate
pred3 = performance(perf, "tpr","fpr")
# 3. Plot the ROC curve
plot(pred3,main="ROC Curve for Random Forest",col=2,lwd=2)
abline(a=0,b=1,lwd=2,lty=2,col="gray")

In [ ]:
# Validation cohort
pred<-predict(rf,test.data)
confusionMatrix(pred,test.data$Disease_stage)

In [ ]:
#Validation AUC
pred1=predict(rf,newdata = test.data,type = "prob")
perf = prediction(pred1[,2],test.data$Disease_stage)
# 1. Area under curve
auc = performance(perf, "auc")
auc <- as.numeric(auc@y.values)
auc
rf$err.rate[nrow(rf$err.rate),1]
# 2. True Positive and Negative Rate
pred3 = performance(perf, "tpr","fpr")
# 3. Plot the ROC curve
plot(pred3,main="ROC Curve for Random Forest",col=2,lwd=2)
abline(a=0,b=1,lwd=2,lty=2,col="gray")